In [ ]:
import MLJ
import DataFrames as DF
import CSV
import StatsBase
import CloudMicrophysics.PreprocessAerosolData as PAD
using DataFramesMeta
using CairoMakie

In [ ]:
include("ReadAerosolDataset.jl")

In [ ]:
function plot_accuracy_scatterplot(dataset_filename, mach_filename=nothing)
    X_test, Y_test, initial_data = read_aerosol_dataset(dataset_filename)
    ARG_act_frac = PreprocessAerosolData.get_ARG_act_frac(X_test)[:,1]
    if mach_filename === nothing
        pred_act_frac = ARG_act_frac
    else
        mach = MLJ.machine(mach_filename)
        @time "Time to predict" pred_act_frac = MLJ.predict(mach, X_test)
        # pred_act_frac = PreprocessAerosolData.get_ARG_act_frac(X_test, MLJ.predict(mach, X_test))[:,1]
    end
    PySDM_act_frac = Y_test
    # PySDM_act_frac = PreprocessAerosolData.get_ARG_act_frac(X_test, Y_test)[:,1]
    println("RMSE: ", sqrt(StatsBase.msd(PySDM_act_frac, pred_act_frac)))
    fig = Figure(resolution=(500, 500))
    ax = Axis(fig[1,1], xlabel="PySDM act frac", ylabel="Predicted act frac")
    scatter!(ax, PySDM_act_frac, pred_act_frac, color=:blue, markersize=10, alpha=0.2)
    lines!(ax, [0, 1], [0, 1], color=:red, label=nothing)
    return fig
end

In [ ]:
plot_accuracy_scatterplot("datasets/4modal_dataset1_test.csv", "emulators/4modal_nn_machine1.jls")